# **<center style="color: green;">Desafío - Transformación y manipulación de datos (Parte II)</center>**

### **Nombre:** Cristian Tobar Morales ###
### **Curso:** Programación con Python para el análisis de datos (G101) ###

### **Descripción**
La **empresa ADL** se dedica a la venta de productos a nivel regional en los Estados Unidos,
con equipos de ventas que operan en diferentes estados. La compañía ha establecido un
sistema de comisiones para incentivar a sus equipos de ventas a preferir la venta de
productos de alto margen antes que los de bajo margen de utilidad.
Como Analista de Datos, le solicitan analizar información sobre las ventas realizadas por
los equipos de ventas de ADL, incluyendo detalles sobre los productos vendidos, los
clientes, las fechas de los pedidos y los plazos de entrega.
Para resolver este desafío deberás cargar datos desde Hojas de Excel del archivo
US_Regional_Sales_Data.xlsx. En base a esto, debe generar una visualización de datos
como la que se muestra en las hojas Reporte1 y Reporte2 del archivo reporte.xlsx.
Para cumplir con lo solicitado, puedes aplicar los siguientes pasos:

1. Carga los datos de cada pestaña del archivo US_Regional_Sales_Data.xlsx en un
DataFrame independiente. (hint: Puedes obtener los nombres de las hojas usando
pd.ExcelFile(archivo).sheet_names)

In [ ]:
# Importar libreria
import pandas as pd

# Ruta del archivo
ar = "../Desafío_Transformación_Manipulación_Datos/db/US_Regional_Sales_Data.xlsx"

# Nombre de de las hojas
nombre_hojas = pd.ExcelFile(ar).sheet_names

#Se crea una diccionario para almacenar los Dataframe
dataframes = {}

#Cargar cada hoja al diccionario
for hoja in nombre_hojas:
    dataframes[hoja] = pd.read_excel(ar, sheet_name=hoja)

# Mostrar los nombres de los DataFrames
for nombre_hoja, df in dataframes.items():
    print(f'DataFrame para la hoja: {nombre_hoja}')

DataFrame para la hoja: Sales Orders Sheet
DataFrame para la hoja: Customers Sheet
DataFrame para la hoja: Store Locations Sheet
DataFrame para la hoja: Products Sheet
DataFrame para la hoja: Sales Team Sheet


2. Cruza todos los DataFrames usando validación many_to_one, y guarda el resultado
en una nueva variable llamada df_base.

In [12]:
# Los dataframes
df_sales_orders = dataframes['Sales Orders Sheet']
df_customers = dataframes['Customers Sheet']
df_store_locations = dataframes['Store Locations Sheet']
df_products = dataframes['Products Sheet']
df_sales_team = dataframes['Sales Team Sheet']

# Cruzar los dataframes
# 1: Se cruza df_sales_orders con df_customers usando 'CustomerID'
df_base = df_sales_orders.merge(df_customers, on = '_CustomerID', validate = 'many_to_one')

# Luego cruzamos el resultado con df_store_locations usando 'StoreID'
df_base = df_base.merge(df_store_locations, on = '_StoreID', validate = 'many_to_one')

# Luego cruzamos el resultado con df_products usando 'ProductID'
df_base = df_base.merge(df_products, on = '_ProductID', validate = 'many_to_one')

# Finalmente, cruzamos el resultado con df_sales_team usando 'SalesPersonID'
df_base = df_base.merge(df_sales_team, on = '_SalesTeamID', validate = 'many_to_one')

# Mostrar los resultados
df_base.head()


,OrderNumber,Sales Channel,WarehouseCode,ProcuredDate,OrderDate,ShipDate,DeliveryDate,CurrencyCode,_SalesTeamID,_CustomerID,...,AreaCode,Population,Household Income,Median Income,Land Area,Water Area,Time Zone,Product Name,Sales Team,Region
0,SO - 000101,In-Store,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-14,2018-06-19,USD,6,15,...,631,213776,68789,80327,135481314,160302131,America/New York,Dining Furniture,Joshua Bennett,Northeast
1,SO - 000102,Online,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-22,2018-07-02,USD,14,20,...,913,186515,74830,72463,194562941,1309517,America/Chicago,Wreaths,Paul Holmes,Midwest
2,SO - 000103,Distributor,WARE-UHY1004,2017-12-31,2018-05-31,2018-06-21,2018-07-01,USD,21,16,...,734,117070,47179,55990,72727009,2254528,America/Detroit,Stemware,Samuel Fowler,Midwest
3,SO - 000104,Wholesale,WARE-NMK1003,2017-12-31,2018-05-31,2018-06-02,2018-06-07,USD,28,48,...,203,130322,49771,37192,48407373,3739487,America/New York,Accessories,Carlos Miller,West
4,SO - 000105,Distributor,WARE-NMK1003,2018-04-10,2018-05-31,2018-06-16,2018-06-26,USD,22,49,...,203,108802,40213,40467,73880017,1086045,America/New York,Candles,Joe Price,Northeast


3. Agrega las siguientes columnas a df_ base:

In [16]:
# Verificar los nombres de las columnas en df_base
print(df_base.columns)


Index(['OrderNumber', 'Sales Channel', 'WarehouseCode', 'ProcuredDate',
       'OrderDate', 'ShipDate', 'DeliveryDate', 'CurrencyCode', '_SalesTeamID',
       '_CustomerID', '_StoreID', '_ProductID', 'Order Quantity',
       'Discount Applied', 'Unit Price', 'Unit Cost', 'Customer Names',
       'City Name', 'County', 'StateCode', 'State', 'Type', 'Latitude',
       'Longitude', 'AreaCode', 'Population', 'Household Income',
       'Median Income', 'Land Area', 'Water Area', 'Time Zone', 'Product Name',
       'Sales Team', 'Region'],
      dtype='object')


In [17]:
# Contar el número de columnas en df_base
num_columnas = len(df_base.columns)
print(f'El DataFrame df_base tiene {num_columnas} columnas.')


El DataFrame df_base tiene 34 columnas.


In [18]:
# Verificar si las columnas son del tipo datetime
print("OrderDate: ", df_base['OrderDate'].dtype)
print("ProcuredDate: ", df_base['ProcuredDate'].dtype)
print("ShipDate: ", df_base['ShipDate'].dtype)
print("DeliveryDate: ", df_base['DeliveryDate'].dtype)

OrderDate:  datetime64[ns]
ProcuredDate:  datetime64[ns]
ShipDate:  datetime64[ns]
DeliveryDate:  datetime64[ns]


In [19]:
# Se agrega la columna ProcurementDays
df_base['ProcurementDays'] = df_base['OrderDate'] - df_base['ProcuredDate']

# Se agrega la columna ShippingDays
df_base['ShippingDays'] = df_base['ShipDate'] - df_base['OrderDate']

# Se agrega la columna DeliveryDays
df_base['DeliveryDays'] = df_base['DeliveryDate'] - df_base['ShipDate']

# Se agrega la columna CustomerDays
df_base['CustomerDays'] = df_base['ShippingDays'] + df_base['DeliveryDays']